In [13]:
import os
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bf
import pickle
import requests
from pymongo import MongoClient

In [7]:
chromedriver = "/Users/El-Razi/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

urls = ['http://genius.com/search?q=jay+z', 'http://genius.com/search?q=eminem',
        'http://genius.com/search?q=nas', 'http://genius.com/search?q=mos+def',
        'http://genius.com/search?q=talib+kweli', 'http://genius.com/search?q=snoop+dogg',
        'http://genius.com/search?q=kanye+west', 'http://genius.com/search?q=lil+wayne',
        'http://genius.com/search?q=camron', 'http://genius.com/search?q=jim+jones',
        'http://genius.com/search?q=jim+jones', 'http://genius.com/search?q=juelz+santana',
        'http://genius.com/search?q=wu+tang+clan', 'http://genius.com/search?q=mobb+deep',
        'http://genius.com/search?q=krs+one', 'http://genius.com/search?q=eric+b+and+rakim',
        'http://genius.com/search?q=grandmaster+flash', 'http://genius.com/search?q=salt+n+pepa',
        'http://genius.com/search?q=foxy+brown', 'http://genius.com/search?q=naughty+by+nature',
        'http://genius.com/search?q=queen+latifah', 'http://genius.com/search?q=dr+dre',
        'http://genius.com/search?q=common', 'http://genius.com/search?q=tupac',
        'http://genius.com/search?q=notorious+big', 'http://genius.com/search?q=big+daddy+kane',
        'http://genius.com/search?q=will+smith', 'http://genius.com/search?q=50+cent',
        'http://genius.com/search?q=big+l', 'http://genius.com/search?q=nwa']

In [4]:
search_list1 = ['Jay-z', 'Eminem', 'Nas', 'Yasiin-bey', 'Black-star', 'Talib-kweli',
                'Snoop-dogg', 'Kanye-west', 'Lil-wayne', 'Camron', 'Jim-jones',
                'Juelz-santana', 'Wu-tang-clan', 'Mobb-deep', 'Prodigy', 'Krs-one',
                'Eric-b-and-rakim', 'Grandmaster-flash', 'Salt-n-pepa', 'Foxy-brown',
                'Naughty-by-nature', 'Queen-latifah', 'Dr-dre', 'Common', '2pac',
                'The-notorious-big', 'Big-daddy-kane', 'Will-smith', '50-cent',
                'Big-l', 'Nwa']
sub_search1 = [i.split('-')[0] for i in search_list1]

In [13]:
master_list = []
driver = webdriver.Chrome(chromedriver)
for url in urls:
    driver.get(url)
    
    # autoscroll
    for i in range(20):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    data = bf(driver.page_source)
    temp_lst = []
    
    for link in data.find_all('li', class_ = 'search_result'):
        temp_lst.append(str(link.a.get('href')))
    master_list.append([url.split('=')[1], temp_lst])
driver.close()

In [16]:
master_list[0][1]

['http://genius.com/Jay-z-holy-grail-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-niggas-in-paris-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-no-church-in-the-wild-lyrics',
 'http://genius.com/Jay-z-part-ii-on-the-run-lyrics',
 'http://genius.com/Jay-z-tom-ford-lyrics',
 'http://genius.com/Jay-z-fuckwithmeyouknowigotit-lyrics',
 'http://genius.com/Jay-z-oceans-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-gotta-have-it-lyrics',
 'http://genius.com/Jay-z-picasso-baby-lyrics',
 'http://genius.com/Jay-z-open-letter-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-otis-lyrics',
 'http://genius.com/Jay-z-99-problems-lyrics',
 'http://genius.com/Jay-z-empire-state-of-mind-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-who-gon-stop-me-lyrics',
 'http://genius.com/Jay-z-heaven-lyrics',
 'http://genius.com/Jay-z-renegade-lyrics',
 'http://genius.com/Jay-z-and-kanye-west-murder-to-excellence-lyrics',
 'http://genius.com/Jay-z-beach-is-better-interlude-lyrics',
 'http://genius.

In [17]:
song_list = sum([j for i,j in master_list],[])

In [18]:
len(song_list)

6155

In [17]:
def weed_out(linksList, container):
    '''
    removes links from passed linksList that are not in container
    '''
    
    return [link for link in linksList if 
            link.split('-')[0].split('m/')[1] in container]

In [21]:
song_list = weed_out(song_list, sub_search1)

In [22]:
len(song_list)

4998

#### URL Pickle

In [23]:
with open('first_urls.pkl', 'wb') as handle:
    pickle.dump(song_list, handle)

In [24]:
with open('first_urls.pkl', 'rb') as handle:
    song_list = pickle.load(handle)

# Multiple Songs

In [29]:
artists = []
dates = []
songs = []
titles = []

for i in xrange(len(song_list)):
    data = bf(requests.get(song_list[i]).text)
    
    try:
        # artist info (if multiple artists, append list. else append list of solo artist)
        if len(data.find('span', class_="song_info-info").text.encode('ascii','replace').replace('\n','')) > 0:
            artists.append([data.find('a', class_="song_header-primary_info-primary_artist").text.encode('ascii','replace') , 
                            data.find('span', class_="song_info-info").text.encode('ascii','replace').replace('\n','')])
        else:
            artists.append([data.find('a', class_="song_header-primary_info-primary_artist").text.encode('ascii','replace')])
        
        # date info
        dates.append(data.find('release-date').text.encode('ascii','replace').replace('\n','')[12:])
        
        # lyrics info
        song = ''
        for i in range(2, len(data.findAll('a', class_='referent'))):
            song += data.findAll('a', class_='referent')[i].text.encode('ascii','replace').replace(u'\n','. ').replace(u'n\u2019t','nt')
            song += '. '
        import unicodedata
        song = unicodedata.normalize('NFKD', song).encode('ascii', 'ignore')
        songs.append(song)
        
        # title info
        titles.append(u''.join(data.find('h1', class_="song_header-primary_info-title").text).encode('utf-8').strip())
    except:
        pass

In [36]:
rap_corpus = {}
for i in xrange(len(songs)):
    j = {}
    j['artist'] = artists[i]
    j['date'] = dates[i]
    j['lyrics'] = songs[i]
    rap_corpus[titles[i]] = j

In [37]:
len(rap_corpus), rap_corpus

(3274,
 {'Smoke': {'artist': ['Foxy Brown', 'Trackmasters'],
   'date': '',
   'lyrics': "Blue sapphires on days when she feeling Hoover. She hood and in the mood when I'm in the mood. Erotic, so exotic, I'm psychotic about it. I don't want forever, I just wanna taste her love sample. That product, I bet a nigga tongue go numb. She's a narcotic, that bomb shit burning, we smoking. My old flame, my Mary Jane, we got a love thing. She ain't jealous, I keep Nina around. In the small of my back in case some shit go down. Right under my Hermes,. I'm hearing the word is. Me, I'm a P.I.M.P. I let Trey hit some, then Dre hit some. Then pass it to the homies 'til we all get done. Niggas can't comprehend what this shit 'bout. Police coming, you like, put this shit out. You like smoke and you got me high. (You make a nigga want to get down). High. (High feeling like I'm up in a cloud). Girl, what the fuck you done to me?. You got me feeling like you just rolled up for me. You're like smoke and yo

#### Corpus Pickle

In [38]:
with open('first_corp.pkl', 'wb') as handle:
    pickle.dump(rap_corpus, handle)

In [40]:
with open('first_corp.pkl', 'rb') as handle:
    rap_corpus = pickle.load(handle)

In [39]:
#!wc -l *

   12401 GeniusAPI.ipynb
   45896 first_corp.pkl
    9997 first_urls.pkl
      33 test_corp.pkl
   68327 total


## Round 2

In [2]:
urls2 = ['http://genius.com/search?q=fugees', 'http://genius.com/search?q=erykah+badu',
        'http://genius.com/search?q=lauryn+hill', 'http://genius.com/search?q=outkast',
        'http://genius.com/search?q=tribe+called+quest', 'http://genius.com/search?q=andre+3000',
        'http://genius.com/search?q=de+la+soul', 'http://genius.com/search?q=cunninlynguists',
        'http://genius.com/search?q=atmosphere', 'http://genius.com/search?q=aesop+rock',
        'http://genius.com/search?q=gza', 'http://genius.com/search?q=kool+keith',
        'http://genius.com/search?q=canibus', 'http://genius.com/search?q=rza',
        'http://genius.com/search?q=the+roots', 'http://genius.com/search?q=killah+priest',
        'http://genius.com/search?q=ghostface+killah', 'http://genius.com/search?q=blackalicious',
        'http://genius.com/search?q=kool+g+rap', 'http://genius.com/search?q=redman',
        'http://genius.com/search?q=e40', 'http://genius.com/search?q=mf+doom',
        'http://genius.com/search?q=beastie+boys', 'http://genius.com/search?q=das+efx',
        'http://genius.com/search?q=xzibit', 'http://genius.com/search?q=raekwon',
        'http://genius.com/search?q=method+man', 'http://genius.com/search?q=wale',
        'http://genius.com/search?q=busta+rhymes', 'http://genius.com/search?q=tech+n9ne']

In [4]:
search_list2 = ['Fugees', 'Erykah-badu', 'Lauryn-hill', 'Outkast', 'Andre-3000', 'De-la-soul',
                'Cunninlynguists', 'Atmosphere', 'Aesop-rock', 'Gza', 'Kool-keith', 'Canibus',
                'Rza', 'The-roots', 'Killah-priest', 'Ghostface-killah', 'Blackalicious',
                'Kool-g-rap', 'Redman', 'E-40', 'Mf-doom', 'Beastie-boys', 'Das-efx', 'Xzibit',
                'Raekwon', 'Method-man', 'Wale', 'Busta-rhymes', 'Tech-n9ne']
sub_search2 = [i.split('-')[0] for i in search_list2]

In [8]:
master_list2 = []
driver = webdriver.Chrome(chromedriver)
for url in urls2:
    driver.get(url)
    
    # autoscroll
    for i in range(20):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    data2 = bf(driver.page_source)
    temp_lst2 = []
    
    for link in data2.find_all('li', class_ = 'search_result'):
        temp_lst2.append(str(link.a.get('href')))
    master_list2.append([url.split('=')[1], temp_lst2])
driver.close()

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


#### Just checkin'

In [9]:
song_list2 = sum([j for i,j in master_list2],[])

In [19]:
len(song_list2)

4928

In [12]:
for link in xrange(len(song_list2)):
    print link, song_list2[link]

In [18]:
song_list2 = weed_out(song_list2, sub_search2)

#### URL Pickle

In [20]:
with open('second_urls.pkl', 'wb') as handle:
    pickle.dump(song_list2, handle)

In [21]:
with open('second_urls.pkl', 'rb') as handle:
    song_list2 = pickle.load(handle)

#### Scrapin'

In [22]:
artists2 = []
dates2 = []
songs2 = []
titles2 = []

for i in xrange(len(song_list2)):
    data = bf(requests.get(song_list2[i]).text)
    
    try:
        # artist info (if multiple artists, append list. else append list of solo artist)
        if len(data.find('span', class_="song_info-info").text.encode('ascii','replace').replace('\n','')) > 0:
            artists2.append([data.find('a', class_="song_header-primary_info-primary_artist").text.encode('ascii','replace') , 
                            data.find('span', class_="song_info-info").text.encode('ascii','replace').replace('\n','')])
        else:
            artists2.append([data.find('a', class_="song_header-primary_info-primary_artist").text.encode('ascii','replace')])
        
        # date info
        dates2.append(data.find('release-date').text.encode('ascii','replace').replace('\n','')[12:])
        
        # lyrics info
        song = ''
        for i in xrange(2, len(data.findAll('a', class_='referent'))):
            song += data.findAll('a', class_='referent')[i].text.encode('ascii','replace').replace(u'\n','. ').replace(u'n\u2019t','nt')
            song += '. '
        import unicodedata
        song = unicodedata.normalize('NFKD', song).encode('ascii', 'ignore')
        songs2.append(song)
        
        # title info
        titles2.append(u''.join(data.find('h1', class_="song_header-primary_info-title").text).encode('utf-8').strip())
    except:
        pass

ChunkedEncodingError: ("Connection broken: error(54, 'Connection reset by peer')", error(54, 'Connection reset by peer'))

In [25]:
half_corpus = {}
for i in xrange(len(songs2)):
    j = {}
    j['artist'] = artists2[i]
    j['date'] = dates2[i]
    j['lyrics'] = songs2[i]
    half_corpus[titles2[i]] = j

In [24]:
len(titles2)

2982

#### Half Corpus Pickle!

In [26]:
with open('half_corp.pkl', 'wb') as handle:
    pickle.dump(half_corpus, handle)

In [27]:
with open('half_corp.pkl', 'rb') as handle:
    half_corpus = pickle.load(handle)